In [2]:
import pandas as pd
import numpy as np

## Costanzo Data

It is a matrix of gene to gene interactions that is split between three files, ExE, ExN, NxN,
where E stands for essential genes and N for nonessential genes.

In this notebook we load those matrices and combine them into a single big matrix.

In [ ]:
# Gene-gene interactions, they need to be combined, cause separatedly for essential and non-essential genes
GxG_data_Costanzo_ExE = pd.read_csv('data/SGA_ExE_clustered.cdt', sep = '\t', dtype = 'str')
GxG_data_Costanzo_ExN = pd.read_csv('data/SGA_ExN_clustered.cdt', sep = '\t', dtype = 'str')
GxG_data_Costanzo_NxN = pd.read_csv('data/SGA_NxN_clustered.cdt', sep = '\t', dtype = 'str')

First, we extract only the rows and columns corresponding to the genes and we change them to use ORF gene ids.

The matrices are also in a special format, CDT, where the first few rows and first few columns correspond to some different values (GWEIGHT etc.). We remove those.

In [4]:
def cdt_to_mat(cdt):
    mat = cdt.iloc[5:, 6:].apply(pd.to_numeric, errors="coerce")
    mat.columns = cdt.iloc[1, 6:]
    return mat.set_index(cdt["ORF"].iloc[5:])

ExE = cdt_to_mat(GxG_data_Costanzo_ExE)
ExN = cdt_to_mat(GxG_data_Costanzo_ExN)
NxN = cdt_to_mat(GxG_data_Costanzo_NxN)

Some genes are repeated, as we see in the next cell. This corresponds to multiple different strains of the yeast being present in the dataset.

In [5]:
ExE.head()

1,YDR478W,YHR040W,YIL104C,YDR064W,YDL208W,YMR290C,YLL011W,YLR186W,YMR229C,YDR339C,...,YBL034C,YBR055C,YLR105C,YDL103C,YDL103C,YKL104C,YKL024C,YOR074C,YDR081C,YOR204W
ORF,,,,,,,,,,,,,,,,,,,,,
YPR016C,0.065,0.013,-0.009,0.038,0.068,-0.233,-0.015,0.059,0.052,0.047,...,-0.028,-0.049,0.029,0.008,0.031,0.033,-0.093,-0.072,-0.003,-0.033
YPR016C,0.096,0.070,0.010,0.083,0.059,-0.245,-0.038,0.026,0.108,0.079,...,0.029,-0.014,0.026,-0.016,0.128,-0.036,-0.160,0.002,-0.024,-0.017
YBR109C,0.025,-0.005,0.062,0.020,0.044,-0.172,-0.077,0.024,0.107,0.128,...,NaN,NaN,0.040,0.044,0.045,0.107,-0.053,0.038,0.011,-0.010
YBR109C,0.053,0.005,0.143,0.069,-0.025,-0.379,-0.211,0.057,-0.127,0.076,...,NaN,NaN,0.082,0.136,0.082,0.117,-0.257,0.082,-0.138,0.029
YDR280W,-0.074,0.032,-0.044,NaN,0.013,0.036,-0.073,-0.058,-0.005,NaN,...,-0.072,-0.014,-0.003,0.029,0.025,0.008,0.041,0.104,NaN,-0.033


In [6]:
all_genes = sorted(set(ExE.index).union(ExE.columns).union(ExN.index).union(ExN.columns).union(NxN.index).union(NxN.columns))
ExE_ = ExE.groupby(by=ExE.index).mean().T.groupby(by=ExE.columns).mean().T.reindex(index=all_genes, columns=all_genes)
ExN_ = ExN.groupby(by=ExN.index).mean().T.groupby(by=ExN.columns).mean().T.reindex(index=all_genes, columns=all_genes)
NxN_ = NxN.groupby(by=NxN.index).mean().T.groupby(by=NxN.columns).mean().T.reindex(index=all_genes, columns=all_genes)

In [7]:
GxG = ExE_.fillna(0) + ExN_.fillna(0) + NxN_.fillna(0)
GxG[(ExE_.isna() & ExN_.isna() & NxN_.isna())] = np.nan

In [8]:
print(f"{GxG.isnull().sum().sum() / GxG.shape[0]**2 * 100:.2f}% NaN values")

48.80% NaN values


We will drop the columns that have more than 15% of NaNs and also the corresponding rows.

## Result
There are three ways to use the Costanzo data. 

1. Either we use it for evaluation and then we want it as a gene by gene matrix. 
2. Or we use it as training data and then we want it as some features for every gene.
3. Take just the essential/nonessential gene labeling and use it for classification to evaluate our embeddings

In [9]:
# Option 1
GxG.to_parquet("data/Costanzo_GxG.parquet")

In [10]:
# Option 2
threshold = 0.35
Costanzo_features = GxG.loc[:, GxG.columns[GxG.isnull().mean(axis=0) < threshold]]
Costanzo_features = Costanzo_features.fillna(Costanzo_features.mean())
Costanzo_features.to_parquet("data/Costanzo_features.parquet")

In [43]:
# Option 3
essential = set(ExE.index)
nonessential = set(NxN.index)
# For some reason, there is a gene that is both essential and nonessential :D
both = essential.intersection(nonessential)
print("Both essential and nonessential: ", both)
# Let's remove it from both sets
essential -= both
nonessential -= both

labels = pd.DataFrame(index=list(sorted(essential.union(nonessential))))
labels.index.name = "gene_id"
labels["essential"] = 0
labels.loc[list(essential), "essential"] = 1
assert labels["essential"].sum() == len(essential)

labels.to_csv("data/Costanzo_classes.csv")

Both essential and nonessential:  {'YLR268W'}


## Using the GxG data for evaluation
We first load the embeddings that we want to evaluate. Note: They cannot contain the Costanzo data!

In [31]:
emb = pd.read_parquet("data/emb_full_pca.parquet")
emb = emb.set_index("gene_id")

In [32]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

def eval_with_Costanzo_GxG(GxG, emb, subset_size=1000, test_size=0.1):
    best = np.inf
    best_subset = None
    for i in range(100):
        subset = np.random.choice(np.arange(GxG.shape[0]), (subset_size,), replace=False)
        data = GxG.iloc[subset, subset]
        if data.isnull().sum().sum() < best:
            best = data.isnull().sum().sum()
            best_subset = subset

    subset_train, subset_test = train_test_split(best_subset, test_size=test_size)
    train_data = GxG.iloc[subset_train, subset_train]
    test_data = GxG.iloc[subset_test, subset_test]

    def data_to_Xy(data):
        data = data.stack(future_stack=True).dropna()
        gene_pairs = data.index.to_frame().reset_index(drop=True)
        gene_pairs.columns = ["gene1", "gene2"]
        gene_pairs = gene_pairs.assign(interaction=pd.Series(np.array(data)))

        gene_pairs_emb = gene_pairs.merge(emb, left_on="gene1", right_on="gene_id").merge(emb, left_on="gene2", right_on="gene_id").drop(["gene1", "gene2"], axis=1)
        y = gene_pairs_emb.interaction
        X = gene_pairs_emb.drop(["interaction"], axis=1)
        return X, y
    
    train_X, train_y = data_to_Xy(train_data)
    test_X, test_y = data_to_Xy(test_data)
    
    reg = LinearRegression()
    reg.fit(train_X, train_y)

    pred_y = reg.predict(test_X)
    print("Costanzo GxG eval MSE:", mean_squared_error(test_y, pred_y), "R2:", r2_score(test_y, pred_y))

In [33]:
eval_with_Costanzo_GxG(GxG, emb)

Costanzo GxG eval MSE: 0.0023395187161635088 R2: -0.0002659014295331996


In [ ]:
# # Clear columns for ExN Data
# Costanzo_ExN_cleared = GxG_data_Costanzo_ExN.drop(GxG_data_Costanzo_ExN.columns[[0, 1, 3, 4]], axis=1)

# # Drop the columns without GWEIGHT
# Costanzo_ExN_cleared = Costanzo_ExN_cleared.dropna(subset=["GWEIGHT"])

# # Drop the GWEIGHT column, since all remaining GWEIGHTS are the same 
# Costanzo_ExN_cleared = Costanzo_ExN_cleared.drop(columns=["GWEIGHT"])

# # Rename gene_id column and set it as index to match other datasets
# Costanzo_ExN_cleared = Costanzo_ExN_cleared.rename(columns={"ORF": "gene_id"}).set_index("gene_id")

# # Drop the columns with over 15% nan values
# Costanzo_ExN_cleared = Costanzo_ExN_cleared.loc[:, Costanzo_ExN_cleared.isna().sum() <= 1150]

# # Convert the string float values to floats
# Costanzo_ExN_cleared = Costanzo_ExN_cleared.iloc[:, :].apply(pd.to_numeric, errors="coerce")

# # Fill remaining NaN values with the mean of the column (doesnt work since values are strings)
# Costanzo_ExN_cleared = Costanzo_ExN_cleared.fillna(Costanzo_ExN_cleared.mean())

In [ ]:
# # Clear columns for NxN Data
# Costanzo_NxN_cleared = GxG_data_Costanzo_NxN.drop(GxG_data_Costanzo_NxN.columns[[0, 1, 3, 4]], axis=1)

# # Drop the columns without GWEIGHT
# Costanzo_NxN_cleared = Costanzo_NxN_cleared.dropna(subset=["GWEIGHT"])

# # Drop the GWEIGHT column, since all remaining GWEIGHTS are the same 
# Costanzo_NxN_cleared = Costanzo_NxN_cleared.drop(columns=["GWEIGHT"])

# # Rename gene_id column and set it as index to match other datasets
# Costanzo_NxN_cleared = Costanzo_NxN_cleared.rename(columns={"ORF": "gene_id"}).set_index("gene_id")

# # Drop the columns with over around 15% nan values
# Costanzo_NxN_cleared = Costanzo_NxN_cleared.loc[:, Costanzo_NxN_cleared.isna().sum() <= 550]

# # Convert the string float values to floats
# Costanzo_NxN_cleared = Costanzo_NxN_cleared.iloc[:, :].apply(pd.to_numeric, errors="coerce")

# # Fill remaining NaN values with the mean of the column (doesnt work since values are strings)
# Costanzo_NxN_cleared = Costanzo_NxN_cleared.fillna(Costanzo_NxN_cleared.mean())